In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,286 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,008 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/p


---

## 📘 **PySpark Joins – Agenda**

* Introduction to Joins in PySpark
* Inner Join
* Left Outer Join (Left Join)
* Right Outer Join (Right Join)
* Full Outer Join
* Left Semi Join
* Left Anti Join
* Cross Join (Cartesian Product)
* Self Join
* Multiple Column Joins
* Optimize JOins -> big vs Small and Big vs Big table
* Broadcast Joins
* Shuffle Hash Join
* Sort Merge Join
* Bucketing Starategy To load faster
* Handling Null Values in Joins
* Duplicate Column Names After Joins
* Multiple Table Joins (Chaining Joins)
* Conditional Joins (Non-Equi Joins)
* Join Skew Problems and Solutions
* Salting Technique for Skewed Joins
* Adaptive Query Execution (AQE) for Joins
* Join Hints and Strategies
* Window Functions with Joins
* Aggregations with Joins

---



In [ ]:
result = df1.join(df2,join_condation,join_type)

df1 -> the left df
df2 -> the right df
join_condation : the column(s) to match
join_tyep : the type of join to perform


In [ ]:
# Inner Join

Returns only the rows where thers a match in both tables
Both sides must match, NO match = not in result

In [ ]:
customers = spark.createDataFrame([
    (1, "Alice"), (2, "Bob"), (3, "Charlie")
], ["cust_id", "name"])

customers.show()

+-------+-------+
|cust_id|   name|
+-------+-------+
|      1|  Alice|
|      2|    Bob|
|      3|Charlie|
+-------+-------+



In [ ]:
orders = spark.createDataFrame([
    (101, 1, "Laptop"), (102, 2, "Phone"), (103, 5, "Mouse")
], ["order_id", "cust_id", "product"])

orders.show()


+--------+-------+-------+
|order_id|cust_id|product|
+--------+-------+-------+
|     101|      1| Laptop|
|     102|      2|  Phone|
|     103|      5|  Mouse|
+--------+-------+-------+



In [ ]:
result=orders.join(customers,"cust_id","inner")
result.show()

+-------+--------+-------+-----+
|cust_id|order_id|product| name|
+-------+--------+-------+-----+
|      1|     101| Laptop|Alice|
|      2|     102|  Phone|  Bob|
+-------+--------+-------+-----+



In [ ]:
from ast import Return
Left Outer Join (Left Join)

Returns all rows from the left table and the matched rows from the right table
if there is no match on the right side, null values are used

In [ ]:
result=customers.join(orders,"cust_id","left")
result.show()

+-------+-------+--------+-------+
|cust_id|   name|order_id|product|
+-------+-------+--------+-------+
|      1|  Alice|     101| Laptop|
|      3|Charlie|    NULL|   NULL|
|      2|    Bob|     102|  Phone|
+-------+-------+--------+-------+



In [ ]:
# Right Outer Join (Right Join)

returns all rows from the right table and the matched rows from the left table
if there is no match on the left side, null values are used

In [ ]:
result=customers.join(orders,"cust_id","right")
result.show()

+-------+-----+--------+-------+
|cust_id| name|order_id|product|
+-------+-----+--------+-------+
|      1|Alice|     101| Laptop|
|      5| NULL|     103|  Mouse|
|      2|  Bob|     102|  Phone|
+-------+-----+--------+-------+



In [ ]:
# Full Outer Join

Returns all rows

In [ ]:
result=customers.join(orders,"cust_id","full")
result.show()

+-------+-------+--------+-------+
|cust_id|   name|order_id|product|
+-------+-------+--------+-------+
|      1|  Alice|     101| Laptop|
|      2|    Bob|     102|  Phone|
|      3|Charlie|    NULL|   NULL|
|      5|   NULL|     103|  Mouse|
+-------+-------+--------+-------+



In [ ]:
# Left Semi Join

left semi  join returns all rows from the left table and only
the matched rows from the right table, but it does not include
any data from the right table in the result -> it acts like
a filter
# if you want a unqiue left table rows then we can use left semi join

In [ ]:
customers = spark.createDataFrame([
    (1, "Alice"), (2, "Bob")
], ["cust_id", "name"])

orders = spark.createDataFrame([
    (101, 1, "Laptop"), (102, 1, "Mouse"), (103, 2, "Phone")
], ["order_id", "cust_id", "product"])

print("LEFT SEMI JOIN:")
customers.join(orders, "cust_id", "left_semi").show()

LEFT SEMI JOIN:
+-------+-----+
|cust_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [ ]:
customers.join(orders, "cust_id", "left").show()

+-------+-----+--------+-------+
|cust_id| name|order_id|product|
+-------+-----+--------+-------+
|      1|Alice|     102|  Mouse|
|      1|Alice|     101| Laptop|
|      2|  Bob|     103|  Phone|
+-------+-----+--------+-------+



In [ ]:
# Left anti join

Left anti join is the opposite of left semi join.
It returns all rows from the left table where there is no match in the right table.
its the exact opposit of left semi join  ->

it finds whats missing

In [ ]:
customers = spark.createDataFrame([
    (1, "Alice"), (2, "Bob"), (3, "Charlie"), (4, "David")
], ["cust_id", "name"])

# Orders DataFrame (RIGHT table - used only for filtering)
orders = spark.createDataFrame([
    (101, 1, "Laptop"), (102, 2, "Phone"), (103, 1, "Mouse")
], ["order_id", "cust_id", "product"])

customers.show()
orders.show()

result = customers.join(orders, "cust_id", "left_anti")
result.show()

result1 = customers.join(orders, "cust_id", "left_semi")
result1.show()

+-------+-------+
|cust_id|   name|
+-------+-------+
|      1|  Alice|
|      2|    Bob|
|      3|Charlie|
|      4|  David|
+-------+-------+

+--------+-------+-------+
|order_id|cust_id|product|
+--------+-------+-------+
|     101|      1| Laptop|
|     102|      2|  Phone|
|     103|      1|  Mouse|
+--------+-------+-------+

+-------+-------+
|cust_id|   name|
+-------+-------+
|      3|Charlie|
|      4|  David|
+-------+-------+

+-------+-----+
|cust_id| name|
+-------+-----+
|      1|Alice|
|      2|  Bob|
+-------+-----+



In [ ]:
# Cross Join

Returns the Cartesian product of the two DataFrames
#

In [ ]:
sizes = spark.createDataFrame([
    ("Small",), ("Medium",), ("Large",)
], ["size"])

colors = spark.createDataFrame([
    ("Red",), ("Blue",)
], ["color"])


sizes.show()
colors.show()

result = sizes.crossJoin(colors)
result.show()


+------+
|  size|
+------+
| Small|
|Medium|
| Large|
+------+

+-----+
|color|
+-----+
|  Red|
| Blue|
+-----+

+------+-----+
|  size|color|
+------+-----+
| Small|  Red|
| Small| Blue|
|Medium|  Red|
| Large|  Red|
|Medium| Blue|
| Large| Blue|
+------+-----+



In [ ]:
# self JOin

is when you jin a table with itself, you basicllay treat hte samae table as two
seperate tables


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("SelfJoin").getOrCreate()

# Create employees DataFrame
employees = spark.createDataFrame([
    (1, "Alice", None),
    (2, "Bob", 1),
    (3, "Charlie", 1),
    (4, "David", 2)
], ["emp_id", "emp_name", "manager_id"])

# Create aliases
emp = employees.alias("emp")
mgr = employees.alias("mgr")

# Self Join using col() with qualified names
result = emp.join(
    mgr,
    col("emp.manager_id") == col("mgr.emp_id"),
    "left"
).select(
    col("emp.emp_id").alias("employee_id"),
    col("emp.emp_name").alias("employee_name"),
    col("mgr.emp_name").alias("manager_name")
)

result.show()

+-----------+-------------+------------+
|employee_id|employee_name|manager_name|
+-----------+-------------+------------+
|          1|        Alice|        NULL|
|          2|          Bob|       Alice|
|          3|      Charlie|       Alice|
|          4|        David|         Bob|
+-----------+-------------+------------+



In [ ]:
Multiple Column Joins

when you join tables using more than one column as the
match condations

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MultiColumnJoin").getOrCreate()

# Sales DataFrame
sales = spark.createDataFrame([
    (1, 101, "Laptop", 5),
    (1, 102, "Mouse", 10),
    (2, 101, "Laptop", 3),
    (2, 103, "Keyboard", 7)
], ["store_id", "product_id", "product_name", "quantity_sold"])

# Inventory DataFrame
inventory = spark.createDataFrame([
    (1, 101, 50),
    (1, 102, 100),
    (2, 101, 30),
    (3, 101, 20)
], ["store_id", "product_id", "stock"])

# Multiple Column Join - Match on BOTH store_id AND product_id
result = sales.join(inventory, ["store_id", "product_id"], "inner")
result.show()


+--------+----------+------------+-------------+-----+
|store_id|product_id|product_name|quantity_sold|stock|
+--------+----------+------------+-------------+-----+
|       1|       101|      Laptop|            5|   50|
|       1|       102|       Mouse|           10|  100|
|       2|       101|      Laptop|            3|   30|
+--------+----------+------------+-------------+-----+

